<a href="https://colab.research.google.com/github/Lemito66/Maestria/blob/master/Clasificaci%C3%B3n_clientes_Agency_Cargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from datetime import date

In [ ]:
sales = pd.read_excel('Agency Cargo 2022Rev.xlsx')

# Preparación de los datos

In [ ]:
# Me aseguro que las fechas estén en formato de fecha.
sales['AWB Date'] = pd.to_datetime(sales['AWB Date'])

sales.sort_values(by='AWB Date', ascending = False).head()

,IT,Cod aerolínea,AWB No. SP,Wgt Charge,Tax AWB*,Due Carrier,Wgt Charge.1,Due Agent,AWB Date,Mes,...,BXS,PCS,Neto,Volumen,RATE,ID CLIENTE,Due Agent.1,AWB,Handling,Total AWB
6119,6120,235,235-6136 1005,8971.799805,0,35.000000,0.0,0.0,2022-12-31,12,...,52.00,113,2296,2361,3.80,21080321757996,12.5,9019.299805,0,9019.299805
6117,6118,235,235-6136 0983,8611.610352,0,35.000000,0.0,0.0,2022-12-31,12,...,46.25,98,2783,2861,3.01,DMCC182926,59.0,8705.610352,0,8705.610352
6749,6750,369,369-8604 1616,454.079987,0,113.000000,0.0,0.0,2022-12-31,12,...,4.00,8,159,176,2.58,91244343806,20.0,587.080017,0,587.080017
6748,6749,369,369-8604 1605,831.299988,0,188.000000,0.0,0.0,2022-12-31,12,...,7.00,23,326,326,2.55,1725765406001,49.0,1068.300049,0,1068.300049
2846,2847,145,145-0955 4031,2691.209961,0,224.059998,0.0,0.0,2022-12-31,12,...,14.25,28,762,823,3.27,1791405390001,99.0,3014.270020,0,3014.270020


# Generación de las estadísticas

In [ ]:
# Calcular las estadisticas: Fecha de la última compra de cada cliente (Recency), número de meses con compra por cliente (Frequency), suma de las ventas por cada cliente (Monetary).
sales_stats = sales.pivot_table(values=['AWB Date', 'Mes', 'Total AWB'], index= ['ID CLIENTE'], aggfunc={'AWB Date': max, 'Mes': pd.Series.nunique, 'Total AWB': sum})

# Ordenar por la columna 'Total AWB' en orden descendente (mayor a menor)
sales_stats = sales_stats.sort_values(by='Total AWB', ascending=False)

print(sales_stats)

                 AWB Date  Mes     Total AWB
ID CLIENTE                                  
5003103483     2022-12-30   12  7.458350e+06
21080321757996 2022-12-31   12  3.091667e+06
NL858608650B01 2022-12-31   12  2.157416e+06
01306202210202 2022-12-31    6  1.627807e+06
1712385        2022-12-29   12  1.298624e+06
...                   ...  ...           ...
110105311115   2022-04-10    2  2.942660e+03
1791265009001  2022-11-13    2  2.850840e+03
18697021716853 2022-05-22    1  1.580400e+03
0590060240001  2022-11-21    1  5.711000e+02
C2043234600    2022-01-11    1  5.091000e+02

[100 rows x 3 columns]


## Mejorar la lectura del DataFrame

Para una mejor visualización del dataframe y lo que significa cada columna voy a cambiarlas de la siguiente forma:

1. Agregar una nueva columna llamada "R" que tenga el número de días que han pasado desde su última compra en "Order Date"
2. Renombrar "Order ID" a "F".
3. Renombrar "Sales" a "M".
4. Remover la columna "Order Date" que ya no necesito.

In [ ]:
# Transformar la "fecha de la ultima compra" a "dias desde la ultima compra"
FREQUENCY = 'F'
RECENCY = 'R'
MONETARY = 'M'
sales_frm = sales_stats.copy()

today = sales['AWB Date'].max()
sales_frm[RECENCY] = (today - sales_frm['AWB Date']).dt.days

# Renombrar columnas para mejor lectura y remover las que ya no necesito
sales_frm = sales_frm.rename(columns={"Mes": FREQUENCY, "Total AWB": MONETARY})
sales_frm = sales_frm.drop(columns=['AWB Date'])

print(sales_frm)
print(type(sales_frm))

                 F             M    R
ID CLIENTE                           
5003103483      12  7.458350e+06    1
21080321757996  12  3.091667e+06    0
NL858608650B01  12  2.157416e+06    0
01306202210202   6  1.627807e+06    0
1712385         12  1.298624e+06    2
...             ..           ...  ...
110105311115     2  2.942660e+03  265
1791265009001    2  2.850840e+03   48
18697021716853   1  1.580400e+03  223
0590060240001    1  5.711000e+02   40
C2043234600      1  5.091000e+02  354

[100 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
#Prueba para visualizar las fechas del cliente con compras más recientes
sales[sales['ID CLIENTE'] == '110105311115'].sort_values(by='AWB Date', ascending=False).head()
cliente_especifico = customers_scores_df.loc[customers_scores_df.index == '110105311115']
print(cliente_especifico)

              F            M    R Fscore Mscore Rscore ScoreTOTAL
ID CLIENTE                                                       
110105311115  2  2942.660034  265      D      D      D        DDD


# Calificación de clientes

Por cada columna F, R y M de los clientes se puede dar una calificación de la 'A' a la 'D' donde 'A' puede ser por ejemplo un cliente más fidelizado  y 'D' es un cliente menos fidelizado.


## Funciones de ayuda
Estas son tres funciones que permiten generar una calificacón por cada columna F, R y M de un dataframe.

In [ ]:
# Función que ayuda a saber si un número está dentro de un rango o no.
def between(value, min_limit, max_limit):
  return (value >= min_limit) & (value < max_limit)

In [ ]:
# Función que ayuda a dar un score a cualquier columna de un FRM dataframe
def add_score_for(df, column, reverse=False):
  Q1 = df[column].quantile(0.25)
  Q2 = df[column].quantile(0.50)
  Q3 = df[column].quantile(0.75)

  print(f'Q1 de {column}: {Q1}')
  print(f'Q2 de {column}: {Q2}')
  print(f'Q3 de {column}: {Q3}')

  min_limit = df[column].min()
  max_limit = df[column].max() + 1

  print(f'min {column}: {min_limit}')
  print(f'max {column}: {max_limit}')

  scale = ['A', 'B', 'C', 'D']

  if (reverse):
    scale.reverse()

  df.loc[between(df[column], min_limit, Q1), column + 'score'] = scale[3]
  df.loc[between(df[column], Q1, Q2), column + 'score'] = scale[2]
  df.loc[between(df[column], Q2, Q3), column + 'score'] = scale[1]
  df.loc[between(df[column], Q3, max_limit), column + 'score'] = scale[0]

#  df.loc[between(df[column], Q3, max_limit), column + 'score'] = scale[0]
#  df.loc[between(df[column], Q2, Q3), column + 'score'] = scale[1]
#  df.loc[between(df[column], Q1, Q2), column + 'score'] = scale[2]
#  df.loc[between(df[column], min_limit, Q1), column + 'score'] = scale[3]

In [ ]:
def score_customers(df):
  score_df = df.copy()
  add_score_for(score_df, FREQUENCY)
  add_score_for(score_df, MONETARY)
  add_score_for(score_df, RECENCY, True)

  return score_df


Ahora puedo usar cualquier dataframe que tenga las columnas F, R y M y obtener la calificación de los clientes.

In [ ]:
customers_scores_df = score_customers(sales_frm)
print (customers_scores_df)

Q1 de F: 3.0
Q2 de F: 7.0
Q3 de F: 10.0
min F: 1
max F: 13
Q1 de M: 40983.797775268555
Q2 de M: 111704.0947265625
Q3 de M: 365367.0651855469
min M: 509.1000061035156
max M: 7458350.801086426
Q1 de R: 1.0
Q2 de R: 8.5
Q3 de R: 153.0
min R: 0
max R: 355
                 F             M    R Fscore Mscore Rscore
ID CLIENTE                                                
5003103483      12  7.458350e+06    1      A      A      B
21080321757996  12  3.091667e+06    0      A      A      A
NL858608650B01  12  2.157416e+06    0      A      A      A
01306202210202   6  1.627807e+06    0      C      A      A
1712385         12  1.298624e+06    2      A      A      B
...             ..           ...  ...    ...    ...    ...
110105311115     2  2.942660e+03  265      D      D      D
1791265009001    2  2.850840e+03   48      D      D      C
18697021716853   1  1.580400e+03  223      D      D      D
0590060240001    1  5.711000e+02   40      D      D      C
C2043234600      1  5.091000e+02  354   

# Interpretación de los datos
Esto es solo para probar qué información se puede sacar del dataframe con resultados, no tiene ningún objetivo en particular.

In [ ]:
def describe_customer_scores(customers_score):
  F = FREQUENCY + 'score'
  R = RECENCY + 'score'
  M = MONETARY + 'score'
  total = len(customers_score)

  clientes_AAA = customers_score[(customers_score[F] == 'A') & (customers_score[R] == 'A') & (customers_score[M] == 'A')]
  clientes_DDD = customers_score[(customers_score[F] == 'D') & (customers_score[R] == 'D') & (customers_score[M] == 'D')]

  print (f'{round(len(clientes_AAA) * 100 / total, 2)}% son clientes AAA.')
  print (f'{round(len(clientes_DDD) * 100 / total, 2)}% son clientes DDD.')


In [ ]:
describe_customer_scores(customers_scores_df)

7.0% son clientes AAA.
8.0% son clientes DDD.


In [ ]:
# Clientes que gastan y venian bastantes veces, pero dejaron de venir.
foo = customers_scores_df[(customers_scores_df['Fscore'] == 'A') & (customers_scores_df['Mscore'] == 'A') & (customers_scores_df['Rscore'].isin(['C', 'D']))]
print(len(foo))


1


In [ ]:
# Clientes que gastan, vienen poco, vinieron recien (Clientes nuevos) -> Fidelizar.
foo = customers_scores_df[(customers_scores_df['Fscore'].isin(['C', 'D'])) & (customers_scores_df['Mscore'] == 'A') & (customers_scores_df['Rscore'].isin(['A', 'B']))]
print(len(foo))
print(foo)


1
                F             M  R Fscore Mscore Rscore
ID CLIENTE                                             
01306202210202  6  1.627807e+06  0      C      A      A


# PRUEBAS

PREUBAS PARA AJUSTA LOS SCORES.

In [ ]:
# Función para asignar puntuaciones personalizadas a una columna
def add_score_for(df, column, q_values, reverse=False):
    min_limit = df[column].min()
    max_limit = df[column].max() + 1

    scale = ['A', 'B', 'C', 'D']
    if reverse:
        scale.reverse()

    df[f'{column}score'] = 'D'  # Inicializa con la peor puntuación 'D'

    q1, q2, q3 = q_values

    condition = (df[column] >= q3) & (df[column] <= max_limit)
    df.loc[condition, f'{column}score'] = scale[0]

    condition = (df[column] >= q2) & (df[column] < q3)
    df.loc[condition, f'{column}score'] = scale[1]

    condition = (df[column] >= q1) & (df[column] < q2)
    df.loc[condition, f'{column}score'] = scale[2]

    condition = (df[column] >= min_limit) & (df[column] < q1)
    df.loc[condition, f'{column}score'] = scale[3]

# Función para puntuar a los clientes
def score_customers(df, q_values):
    score_df = df.copy()
    for column, q_values_column, reverse in q_values:
        add_score_for(score_df, column, q_values_column, reverse)
    return score_df

# Tu DataFrame de ventas
sales_frm

# Definición de los valores de q para cada columna
q_values = [
    ('F', (df['F'].quantile(0.25), df['F'].quantile(0.50), df['F'].quantile(0.75)), False),
    ('M', (df['M'].quantile(0.25), df['M'].quantile(0.50), df['M'].quantile(0.75)), False),
    ('R', (30, 60, 90), True)
]

# Asignar las puntuaciones personalizadas
customers_scores_df = score_customers(sales_frm, q_values)
print(customers_scores_df)
print(q_values)

                 F             M    R Fscore Mscore Rscore
ID CLIENTE                                                
5003103483      12  7.458350e+06    1      A      A      A
21080321757996  12  3.091667e+06    0      A      A      A
NL858608650B01  12  2.157416e+06    0      A      A      A
01306202210202   6  1.627807e+06    0      C      A      A
1712385         12  1.298624e+06    2      A      A      A
...             ..           ...  ...    ...    ...    ...
110105311115     2  2.942660e+03  265      D      D      D
1791265009001    2  2.850840e+03   48      D      D      B
18697021716853   1  1.580400e+03  223      D      D      D
0590060240001    1  5.711000e+02   40      D      D      B
C2043234600      1  5.091000e+02  354      D      D      D

[100 rows x 6 columns]
[('F', (3.0, 7.0, 10.0), False), ('M', (40983.797775268555, 111704.0947265625, 365367.0651855469), False), ('R', (30, 60, 90), True)]


In [ ]:
customers_scores_df['ScoreTOTAL'] = customers_scores_df['Fscore']+customers_scores_df['Mscore']+customers_scores_df['Rscore']
print(customers_scores_df)
print(type(customers_scores_df))

                 F             M    R Fscore Mscore Rscore ScoreTOTAL
ID CLIENTE                                                           
5003103483      12  7.458350e+06    1      A      A      A        AAA
21080321757996  12  3.091667e+06    0      A      A      A        AAA
NL858608650B01  12  2.157416e+06    0      A      A      A        AAA
01306202210202   6  1.627807e+06    0      C      A      A        CAA
1712385         12  1.298624e+06    2      A      A      A        AAA
...             ..           ...  ...    ...    ...    ...        ...
110105311115     2  2.942660e+03  265      D      D      D        DDD
1791265009001    2  2.850840e+03   48      D      D      B        DDB
18697021716853   1  1.580400e+03  223      D      D      D        DDD
0590060240001    1  5.711000e+02   40      D      D      B        DDB
C2043234600      1  5.091000e+02  354      D      D      D        DDD

[100 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
customers_scores_df.groupby('ScoreTOTAL')['ID CLIENTE'].nunique()

KeyError: ignored

In [ ]:
#Prueba para visualizar las fechas del cliente con compras más recientes
sales[sales['ID CLIENTE'] == '9715236882'].sort_values(by='AWB Date', ascending=False).head()
cliente_especifico = customers_scores_df.loc[customers_scores_df.index == '9715236882']
print(cliente_especifico)

            F              M   R Fscore Mscore Rscore
ID CLIENTE                                           
9715236882  9  587422.739624  94      B      A      D


In [ ]:
def describe_customer_scores(customers_score):
  F = FREQUENCY + 'score'
  R = RECENCY + 'score'
  M = MONETARY + 'score'
  total = len(customers_score)

  clientes_AAA = customers_score[(customers_score[F] == 'A') & (customers_score[R] == 'A') & (customers_score[M] == 'A')]
  clientes_DDD = customers_score[(customers_score[F] == 'D') & (customers_score[R] == 'D') & (customers_score[M] == 'D')]

  print (f'{round(len(clientes_AAA) * 100 / total, 2)}% son clientes AAA.')
  print (f'{round(len(clientes_DDD) * 100 / total, 2)}% son clientes DDD.')


In [ ]:
describe_customer_scores(customers_scores_df)

13.0% son clientes AAA.
9.0% son clientes DDD.


In [ ]:
# Ejemplo de un DataFrame con columnas F, M y R
df = sales_frm

# Define una función para asignar puntajes personalizados a una columna
def assign_custom_scores(df, column, q1, q2, q3, reverse=False):
    scale = ['A', 'B', 'C', 'D']
    df[f'{column}score'] = pd.cut(df[column], bins=[-float("inf"), q1, q2, q3, float("inf")], labels=scale)

    if (reverse):
      scale.reverse()

# Valores de cuartiles personalizados para cada columna
q1_F = 3
q2_F = 7
q3_F = 10

q1_M = 40983.797775268555
q2_M = 111704.0947265625
q3_M = 365367.0651855469

q1_R = 30
q2_R = 60
q3_R = 90

# Asigna puntajes personalizados para cada columna
assign_custom_scores(df, 'F', q1_F, q2_F, q3_F)
assign_custom_scores(df, 'M', q1_M, q2_M, q3_M)
assign_custom_scores(df, 'R', q1_R, q2_R, q3_R)

print(df)

                 F             M    R Fscore Mscore Rscore
ID CLIENTE                                                
5003103483      12  7.458350e+06    1      D      D      A
21080321757996  12  3.091667e+06    0      D      D      A
NL858608650B01  12  2.157416e+06    0      D      D      A
01306202210202   6  1.627807e+06    0      B      D      A
1712385         12  1.298624e+06    2      D      D      A
...             ..           ...  ...    ...    ...    ...
110105311115     2  2.942660e+03  265      A      A      D
1791265009001    2  2.850840e+03   48      A      A      B
18697021716853   1  1.580400e+03  223      A      A      D
0590060240001    1  5.711000e+02   40      A      A      B
C2043234600      1  5.091000e+02  354      A      A      D

[100 rows x 6 columns]


In [ ]:
# Clientes con alta frecuencia pero baja recency y baja consumo monetario
# No vienen ultimamente y no compran. :/
foo = customers_scores_df[(customers_scores_df['Mscore'].isin(['D'])) & (customers_scores_df['Rscore'].isin(['D']))]
print(len(foo))
print(foo)

13
                      F             M    R Fscore Mscore Rscore
ID CLIENTE                                                     
1792062616001         5  37430.929993  223      C      D      D
692173979             4  36167.309631  259      C      D      D
NL818325835B01        2  30878.759888  321      D      D      D
1791262212001         4  28103.750000  267      C      D      D
1792130034001         3  27177.220337  286      D      D      D
1728025725001         1  18623.300049  335      D      D      D
USCI9131011559976692  2   7717.200195  330      D      D      D
11459205335           2   7273.879883  229      D      D      D
CZ06550690            3   5174.130005  204      D      D      D
454408846             1   3301.699951  267      D      D      D
110105311115          2   2942.660034  265      D      D      D
18697021716853        1   1580.399994  223      D      D      D
C2043234600           1    509.100006  354      D      D      D


In [ ]:
def add_score_custom2(df, column, q1, q2, q3, reverse=False):
    min_limit2 = df[column].min()
    max_limit2 = df[column].max() + 1

    print(f'min {column}: {min_limit2}')
    print(f'max {column}: {max_limit2}')

    scale = ['A', 'B', 'C', 'D']

    if (reverse):
        scale.reverse()

    df.loc[between(df[column], min_limit2, q1), column + 'score'] = scale[3]
    df.loc[between(df[column], q1, q2), column + 'score'] = scale[2]
    df.loc[between(df[column], q2, q3), column + 'score'] = scale[1]
    df.loc[between(df[column], q3, max_limit2), column + 'score'] = scale[0]

In [ ]:
# Definir los valores de cuartiles personalizados para cada columna
custom_q1_F = 3  # Reemplaza con el valor Q1 deseado para la columna FREQUENCY
custom_q2_F = 7  # Reemplaza con el valor Q2 deseado para la columna FREQUENCY
custom_q3_F = 10  # Reemplaza con el valor Q3 deseado para la columna FREQUENCY

custom_q1_M = 5000  # Reemplaza con el valor Q1 deseado para la columna MONETARY
custom_q2_M = 111704.0947265625  # Reemplaza con el valor Q2 deseado para la columna MONETARY
custom_q3_M = 20000  # Reemplaza con el valor Q3 deseado para la columna MONETARY

custom_q1_R = 10  # Reemplaza con el valor Q1 deseado para la columna RECENCY
custom_q2_R = 20  # Reemplaza con el valor Q2 deseado para la columna RECENCY
custom_q3_R = 31  # Reemplaza con el valor Q3 deseado para la columna RECENCY



def score_customers2(df):
  score_df2 = df.copy()
  add_score_for2(score_df2, FREQUENCY, custom_q1_F, custom_q2_F, custom_q3_F)
  add_score_for2(score_df2, MONETARY, custom_q1_M, custom_q2_M, custom_q3_M)
  add_score_for2(score_df2, RECENCY, custom_q1_R, custom_q2_R, custom_q3_R, True)

  return score_df2

In [ ]:
customers_scores_df2 = score_customers2(sales_frm)
print (customers_scores_df2)

NameError: ignored